In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import time
from IPython import display

from dedalus import public as de
from dedalus.extras import flow_tools

import logging
logger = logging.getLogger(__name__)

In [ ]:
Lx, Ly = (18, 29)
nx, ny = (300, 450)

# Create bases and domain
x_basis = de.Fourier('x', nx, interval=(0, Lx), dealias=3/2)
y_basis = de.Chebyshev('y', ny, interval=(0, Ly), dealias=3/2)
domain = de.Domain([x_basis, y_basis], grid_dtype=np.float64)

In [ ]:
def grow_fun(A,B):
    F = (A.data)*(1-B.data)
    F[A.data < 1e-3] = 0
    return F


def grow_operator(field1,field2):
    return de.operators.GeneralFunction(
        field1.domain,
        layout = 'g',
        func = grow_fun,
        args = (field1,field2,)
    )

de.operators.parseables['grow'] = grow_operator

In [ ]:
params = np.load('Parameters.npz')
Diff = params['Diff'].flat[0]
K =  params['K'].flat[0]
g = params['g'].flat[0]

problem = de.IVP(domain, variables=['rho1','rho2','rho1y','rho2y','rho12'])
problem.parameters['D1'] = 2*Diff
problem.parameters['D2'] = 0.5*Diff
problem.parameters['K'] = K
problem.parameters['g1'] = g
problem.parameters['g2'] = g


problem.substitutions['rho_b'] = "rho1+rho2+rho12" 
problem.substitutions["Lap(A,Ay)"] = "dx(dx(A)) + dy(Ay)"


problem.add_equation("dt(rho1)  - D1*Lap(rho1,rho1y) = g1*grow(rho1,rho_b)  - K*rho1*rho2")
problem.add_equation("dt(rho2)  - D2*Lap(rho2,rho2y) = g2*grow(rho2,rho_b)  - K*rho1*rho2")
problem.add_equation("dt(rho12)  = 2*K*rho1*rho2")
problem.add_equation("rho1y - dy(rho1)  = 0")
problem.add_equation("rho2y - dy(rho2)  = 0")

problem.add_bc("left(rho1y) = 0")
problem.add_bc("left(rho2) = 0")
problem.add_bc("right(rho2) = 0")
problem.add_bc("right(rho1) = 0")

In [ ]:
ts = de.timesteppers.RK443
solver =  problem.build_solver(ts)

In [ ]:
x = domain.grid(0)
y = domain.grid(1)
rho1 = solver.state['rho1']
rho2 = solver.state['rho2']

def r(x,y,x0,y0):
    return np.sqrt((x-x0)**2+(y-y0)**2)

def blob(x,y,x0,y0,r0,w):
    return 0.5*(1 - np.tanh((r(x,y,x0,y0)-r0)/w))

r0 = 0.75
w = 0.22

rho1['g'] = blob(x,y,Lx*0.5,6,r0 + 80*3.04/60,w) # speed is 3.04/60 mm/min
rho2['g'] = blob(x,y,Lx*0.5,15,r0,w) # slow one starts growing ~80 mins after fast one

In [ ]:
solver.stop_sim_time = 420
dt = 0.18*Lx/nx

In [ ]:
analysis = solver.evaluator.add_file_handler('diff_grow', sim_dt=2, max_writes=500)
analysis.add_task('rho1')
analysis.add_task('rho2')
analysis.add_task('rho12')

In [ ]:
logger.info('Starting loop')
start_time = time.time()
while solver.ok:
    solver.step(dt)
    if solver.iteration % 50 == 0:
        logger.info('Iteration: %i, Time: %e, dt: %e' %(solver.iteration, solver.sim_time, dt))

In [ ]:
from dedalus.tools import post
post.merge_process_files("diff_grow", cleanup=True)